In [ ]:
drive.mount('/content/drive')

In [ ]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

dataset_folder = '/content/drive/MyDrive/dipcancer'
benign_folder = os.path.join(dataset_folder, '/content/drive/MyDrive/dipcancer/benign')
malignant_folder = os.path.join(dataset_folder, '/content/drive/MyDrive/dipcancer/malignant')
normal_folder = os.path.join(dataset_folder, '/content/drive/MyDrive/dipcancer/normal')

combined_output_folder = '/content/combined_images'
os.makedirs(combined_output_folder, exist_ok=True)

def preprocess_image(image_path):
    gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    equalized_image = cv2.equalizeHist(gray_image)
    binary_image = cv2.adaptiveThreshold(
        equalized_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    isolated_region = cv2.bitwise_and(equalized_image, equalized_image, mask=binary_image)
    return equalized_image, isolated_region

def load_data(folder, label):
    images = []
    labels = []
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        if not file_name.endswith(('.png', '.jpg', '.jpeg')):
            continue
        original_image, _ = preprocess_image(file_path)
        resized_image = cv2.resize(original_image, (128, 128))
        normalized_image = resized_image / 255.0
        images.append(normalized_image)
        labels.append(label)
    return images, labels

malignant_images, malignant_labels = load_data(malignant_folder, label=1)
benign_images, benign_labels = load_data(benign_folder, label=0)
normal_images, normal_labels = load_data(normal_folder, label=2)
images = np.array(malignant_images + benign_images + normal_images)
labels = np.array(malignant_labels + benign_labels + normal_labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred) * 100
print(f"Accuracy of the model: {accuracy:.2f}%")

for file_name in os.listdir(malignant_folder):
    file_path = os.path.join(malignant_folder, file_name)
    if not file_name.endswith(('.png', '.jpg', '.jpeg')):
        continue
    original_image, isolated_image = preprocess_image(file_path)
    combined_image = np.hstack((original_image, isolated_image))
    output_path = os.path.join(combined_output_folder, file_name)
    cv2.imwrite(output_path, combined_image)

print(f"Images have been saved to: {combined_output_folder}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


152/152 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.6411 - loss: 0.7815 - val_accuracy: 0.7255 - val_loss: 0.9775
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.8639 - loss: 0.3741 - val_accuracy: 0.8709 - val_loss: 0.3951
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 238s 1s/step - accuracy: 0.8869 - loss: 0.2941 - val_accuracy: 0.8512 - val_loss: 0.4252
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.8962 - loss: 0.2734 - val_accuracy: 0.8882 - val_loss: 0.3389
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 182s 1s/step - accuracy: 0.8978 - loss: 0.2616 - val_accuracy: 0.8791 - val_loss: 0.2950
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9105 - loss: 0.2376 - val_accuracy: 0.9136 - val_loss: 0.2241
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.9093 - loss: 0.2486 - val_accuracy: 0.9060 - val_loss: 0.2422
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.9107 - loss: 0.2341 - val_accuracy: 0.910

In [ ]:
joblib.dump(model, '/content/cancer_model.pkl')

['/content/cancer_model.pkl']

In [ ]:
!pip install pyngrok joblib flask opencv-python-headless

from flask import Flask, request, jsonify
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
import joblib
from pyngrok import ngrok
from IPython.display import display, HTML

app = Flask(__name__)

model = joblib.load('/content/cancer_model.pkl')

def isolate_cancer(image_path):
    gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    isolated_region = cv2.bitwise_and(gray_image, gray_image, mask=binary_image)
    return gray_image, isolated_region

def process_image(image):
    gray_image, _ = isolate_cancer(image)
    resized_image = cv2.resize(gray_image, (128, 128))
    return resized_image.flatten()

@app.route('/predict', methods=['POST'])
def predict():
    image = request.files['image']
    image_path = os.path.join('/content/uploads', image.filename)
    os.makedirs('/content/uploads', exist_ok=True)
    image.save(image_path)

    image_features = process_image(image_path)
    prediction = model.predict([image_features])
    label = prediction[0]
    prediction_text = "Benign" if label == 0 else "Malignant" if label == 1 else "Normal"
    stage = "Stage 0: Non-cancerous" if label == 0 else "Stage 3-4: Likely invasive" if label == 1 else "No evidence of cancer detected"

    _, isolated_region = isolate_cancer(image_path)
    _, img_encoded = cv2.imencode('.png', isolated_region)
    img_base64 = img_encoded.tobytes()

    return jsonify({
        "prediction": prediction_text,
        "stage": stage,
        "image": img_base64.hex()  # Send image as hex string to solve errors
    })

!ngrok authtoken 2pNnpRSG84JgHnxb2RC4CULFl8E_4HYfd7jBpKZZ3FYfzTMdK
public_url = ngrok.connect(12080)
print(f" * Flask app running at {public_url}")

from threading import Thread
def run_flask():
    app.run(port=12080)

thread = Thread(target=run_flask)
thread.start()

html_code = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Breast Cancer Detection</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f4f4f9;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }}
        .container {{
            background-color: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            text-align: center;
            width: 300px;
        }}
        input {{
            margin: 10px 0;
            display: block;
            width: 100%;
        }}
        button {{
            padding: 10px 20px;
            background-color: #4CAF50;
            color: white;
            border: none;
            cursor: pointer;
            border-radius: 5px;
        }}
        button:hover {{
            background-color: #45a049;
        }}
        img {{
            margin-top: 20px;
            max-width: 100%;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Breast Cancer Detection</h1>
        <input type="file" id="imageInput" accept="image/*">
        <button onclick="submitImage()">Upload and Predict</button>
        <div id="result"></div>
    </div>

    <script>
        function submitImage() {{
            var imageInput = document.getElementById('imageInput');
            var resultDiv = document.getElementById('result');

            if (imageInput.files.length === 0) {{
                resultDiv.innerHTML = "Please select an image.";
                return;
            }}

            var formData = new FormData();
            formData.append("image", imageInput.files[0]);

            fetch('{public_url}/predict', {{
                method: 'POST',
                body: formData
            }})
            .then(response => response.json())
            .then(data => {{
                resultDiv.innerHTML = `
                    <p>Prediction: <strong>${{data.prediction}}</strong></p>
                    <p>Stage: <strong>${{data.stage}}</strong></p>
                    <img src="data:image/png;base64,${{data.image}}" alt="Processed Image">
                `;
            }});
        }}
    </script>
</body>
</html>
"""

# Fix display HTML in Google Colab
display(HTML(html_code))

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


 * Flask app running at NgrokTunnel: "https://bc79-35-236-229-245.ngrok-free.app" -> "http://localhost:12080"


In [ ]:
ngrok.disconnect(public_url)